In [30]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from os import listdir
from PIL import Image
import os
import cv2

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix



In [72]:
path= 'images/IDC_regular_ps50_idx5'
images = []

for root, dirs, files in os.walk(path):
    for file in files:
        #append the file name to the list
        images.append(os.path.join(root,file))


In [73]:
from keras.preprocessing.image import load_img, img_to_array
from keras.preprocessing.image import ImageDataGenerator
from numpy import expand_dims

In [75]:
def load_data(files):
    X = []
    y = []

    for file in files:
        if file.endswith(".png"):
            if(file[-5] == '1'):
                img = load_img(file, target_size = (50,50))
                data = img_to_array(img)
                pixels =data/ 255
                X.append(pixels)
                y.append(1)
                
            elif(file[-5] == '0'):
                img = load_img(file, target_size = (50,50))
                pixels = img_to_array(img)
                pixels /= 255
                X.append(pixels)
                y.append(0)
    return np.stack(X), y

In [46]:
X=np.load("X_sal.npy")
y=np.load("y_sal.npy")

In [47]:
np.count_nonzero(y == 1),np.count_nonzero(y == 0)

(145854, 177954)

In [48]:

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

In [49]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.losses import binary_crossentropy
# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [59]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Convolution2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import BatchNormalization



In [67]:
from tensorflow.keras.applications.resnet50 import ResNet50,preprocess_input

In [68]:
from tensorflow.keras.models import Model


In [56]:
restnet = ResNet50(include_top=False, weights='imagenet', input_shape=(224,224,3))
for layer in restnet.layers:
    layer.trainable = False
restnet.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_11 (InputLayer)           [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_11[0][0]                   
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, 112, 112, 64) 256         conv1[0][0]                      
___________________________________________________________________________________________

In [60]:
model = Sequential()
model.add(ZeroPadding2D((174, 174), input_shape=(50, 50, 3)))
model.add(restnet)
model.add(Flatten())
#model.add(Dense(512, activation='relu'))
#model.add(Dropout(0.5))
#model.add(BatchNormalization())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(BatchNormalization())
model.add(Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
zero_padding2d (ZeroPadding2 (None, 398, 398, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
flatten_5 (Flatten)          (None, 346112)            0         
_________________________________________________________________
dense_24 (Dense)             (None, 128)               44302464  
_________________________________________________________________
dropout_6 (Dropout)          (None, 128)               0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 128)               512       
_________________________________________________________________
dense_25 (Dense)             (None, 1)                

In [61]:
model.compile(optimizer = 'adam', loss = binary_crossentropy, metrics = ['accuracy'])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [69]:
from tensorflow.keras.callbacks import ModelCheckpoint
checkpoint = ModelCheckpoint("Restnet_best_model", monitor='val_acc', verbose=1,
    save_best_only=True, mode='auto', period=1)

In [70]:
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)

In [71]:
M_RES=model.fit(X_train, y_train,validation_data=(X_val, y_val),  epochs=20,callbacks=[checkpoint,es])

Train on 259046 samples, validate on 64762 samples
Epoch 1/20
   928/259046 [..............................] - ETA: 35:59:11 - loss: 0.5368 - acc: 0.7392

KeyboardInterrupt: 

In [1]:
test_loss, test_acc = model.evaluate(X_test,y_test, verbose=2)

print('\nTest accuracy:', test_acc)

In [ ]:
Y_pred = model.predict(X_test)
y_pred = (Y_pred > 0.5).astype(np.int)
confusion_matrix(y_test, y_pred)